<a href="https://colab.research.google.com/github/quocthang0507/tensorflow_text/blob/main/text_classification_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>